# Advent of Code

In [1]:
from matplotlib import pyplot as plt, animation
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D

import numpy as np

%matplotlib notebook

# Upute za preuzimanje zadataka

Posjetiti stranicu *[Advent of Code](https://adventofcode.com)*. Na toj stranici napraviti acconut (moze se koristiti github/email) i kopirati svoj session. Pozvati fju get_aoc_input sa session_str jednakim session-u i danom i godinom zadatka.

In [2]:
session = '53616c7465645f5fa84cdf4fd1e667823de36970c380c49a1373a1ac0d5a4ddf77aeb87eda22f992bac9a4e24fa8295f06c385916e074b59f6ba4ee8e53b2612'

In [3]:
import requests
import os

def get_aoc_input(session_str, day, year, return_output=False):
    
    if os.path.exists(f'./{year}_{day}_input.txt'):
        with open(f'{year}_{day}_input.txt') as f:
            text = [x.strip() for x in f.readlines()]
        if return_output: return text
    else:
        response = requests.get(
            f'https://adventofcode.com/{year}/day/{day}/input',
            headers={'cookie':f'session={session_str}'}
        )
    
        try:
            with open(f'{year}_{day}_input.txt', 'w') as f:
                f.write(response.text.strip())
            if return_output: return [x.strip() for x in response.text.strip().split('\n')]
        except Exception as e:
            print(f'Error: {e.args}')
            return None

## Experimental Emergency Teleportation
*[Link na zadatak](https://adventofcode.com/2018/day/23)*

In [ ]:
import re
from collections import Counter

def parse_line(line):
    return tuple(map(int, re.match(r'pos=<(-?\d+),(-?\d+),(-?\d+)>, r=(\d+)', line).groups()))

def manh_dist(point1, point2):
    return sum(abs(point1[i] - point2[i]) for i in range(3))

inlist = get_aoc_input(session, 23, 2018, return_output=True)
inputs = [parse_line(line) for line in inlist]
c = Counter()
for n1 in inputs:
    for n2 in inputs:
        if manh_dist(n1,n2) <= n1[-1]:
            c[n1] += manh_dist(n1,n2) <= n1[-1]

n = sorted(c.keys(), key=lambda x: x[-1], reverse=True)[0]
print(c[n])

In [ ]:
from z3 import *

def zabs(x):
    return If(x >= 0,x,-x)

(x,y,z) = (Int('x'),Int('y'),Int('z'))
in_ranges = [Int(f'in_range_{i}') for i in range(len(inputs))]
range_count = Int('sum')
dist_from_zero = Int('dist')

o = Optimize()
for i in range(len(inputs)):
    nx,ny,nz,nr = inputs[i]
    o.add(in_ranges[i] == If(zabs(x-nx) + zabs(y-ny) + zabs(z-nz) <= nr, 1, 0))   
o.add(range_count == sum(in_ranges))
o.add(dist_from_zero == zabs(x) + zabs(y) + zabs(z))

In [ ]:
h1 = o.maximize(range_count)
h2 = o.minimize(dist_from_zero)
print(o.check())
print(o.lower(h2), o.upper(h2))

## Lanternfish

*[Link na zadatak](https://adventofcode.com/2021/day/6)*

In [ ]:
from collections import deque
import re
import itertools as it

Dohvatimo input za zadatak i pocnimo :)

In [ ]:
get_aoc_input(session, 6, 2021)

Vidimo da je input jako lijepo formatiran (ako se radi u python-u ;), to se lako obradi uz malo `str`/`np.loadtxt` magije

### Rjesenje pomocu numpy-a

In [ ]:
lines = np.loadtxt('2021_6_input.txt', delimiter=',', dtype="uint32")
fish_timer = np.zeros(9)
time, count = np.unique(lines, return_counts=True)
fish_timer[time] = count

In [ ]:
def memoize_numpy(fish_timer, days):
    gen = np.copy(fish_timer)
    for _ in range(days):
        gen[7] += gen[0]
        gen = np.roll(gen, -1)
    return sum(gen)

In [ ]:
print(memoize_numpy(fish_timer, 80))
print(memoize_numpy(fish_timer, 256))

In [ ]:
def generator(fish_timer, days):
    for i in range(days):
        yield memoize_numpy(fish_timer, i)

y = [x for x in generator(fish_timer, 256)]

fig, ax = plt.subplots()
ax.plot(y)
plt.show()

### Rjesenje pomocu vulgaris python-a :)

In [ ]:
with open('2021_6_input.txt') as f:
    inlist = list(map(int, f.readline().strip().split(',')))
d = deque([0 for _ in range(9)])
for num in inlist:
    d[num] += 1

In [ ]:
def memoize_pure_python(deq, days):
    d = deque([x for x in deq])
    for _ in range(days):
        d[7] += d[0]
        d.rotate(-1)
    return sum(d)

In [ ]:
print(memoize_pure_python(d, 80))
print(memoize_pure_python(d, 256))

## Smoke Basin

*[Link na zadatak](https://adventofcode.com/2021/day/9)*

In [ ]:
inlist = get_aoc_input(session, 9, 2021, return_output=True)
matrix = [[int(x) for x in line] for line in inlist]
matrixes = np.array([[[int(x) for x in line] for line in inlist]])

In [ ]:
def check_low_spot(i,j,inlist,matrix):
    ok = True
    for (x,y) in [(1,0), (0,1), (-1,0), (0,-1)]:
        if -1 < y + j < len(inlist) and -1 < x + i < len(line):
            ok &= int(inlist[y+j][x+i]) > int(l)
    if ok:
        matrix[j][i] = 0
        return matrix, 1+int(l)
    else:
        matrix[j][i] = 6 if matrix[j][i] != 9 else 9
        return None, None

In [ ]:
lows,s = [],0
for j,line in enumerate(inlist):
    for i,l in enumerate(line):
        m, n = check_low_spot(i,j,inlist,matrix)
        if m is not None:
            s += n
            lows.append((i,j))
            matrixes = np.append(matrixes, [m], axis=0)
print(s)

In [ ]:
def basin_size(x,y,inlist):
    seen = set()
    Q = [(x,y)]
    size = 0
    while len(Q) > 0:
        curr = Q.pop(0)
        curx, cury = curr
        if curr in seen:
            continue
        seen.add(curr)
        if 0 <= cury < len(inlist) and 0 <= curx < len(inlist[0]):
            if int(inlist[cury][curx]) != 9:
                size += 1
                Q += [(curx+1,cury), (curx-1,cury), (curx,cury+1), (curx,cury-1)]
    return size

In [ ]:
basins = []
for (x,y) in lows:
    basins.append(basin_size(x,y,inlist))
basins.sort()
print(f'Size of the largest three basins: {basins[-1]}, {basins[-2]}, {basins[-3]}')
print(f'Those numbers multiplied: {basins[-1] * basins[-2] * basins[-3]}')

In [ ]:
fig, ax = plt.subplots()
cax = ax.pcolormesh(matrixes[0, :, :])

In [ ]:
def basin_animate(i):
    cax.set_array(matrixes[i, :, :])
    return cax

anim = animation.FuncAnimation(fig, basin_animate, interval=100, frames=matrixes.shape[0] - 1)
plt.show()

## Jurassic Jigsaw

In [48]:
get_aoc_input(session, 20, 2020)

with open('2020_20_input.txt') as f:
    tiles = {
        int((line := tile.split('\n'))[0].split()[1][:-1]) : np.array([[int(x) for x in y] for y in line[1:]])
        for tile in f.read().replace('.','0').replace('#','1').split('\n\n')
    }

In [49]:
from functools import reduce

edges = {
    k : set.union(*[{tuple(edge), tuple(reversed(edge))} for edge in [v[0], v[-1], v[:,0], v[:,-1]]])
    for k, v in tiles.items()
}
neighbors = {
    k1 : ({k2 for k2, v2 in edges.items() if v1 & v2} - {k1}) for k1,v1 in edges.items()
}
corners = [k for k, v in neighbors.items() if len(v) == 2]

print(reduce(lambda a,b: a * b, corners))

17148689442341


In [50]:
def rotations(tile):
    for _ in range(2):
        for _ in range(4):
            tile = np.rot90(tile)
            yield tile
        tile = np.flip(tile,0)

In [51]:
current = corners[0]
reverse = False
tiles[current] = next(
    tile for tile in rotations(tiles[current]) 
    if {tuple(tile[-1]), tuple(tile[:,-1])}.issubset(
        set.union(*[edges[k] for k in neighbors[current]])
    )
)
chart = []
chart_row = [tiles[current][1:-1,1:-1]]

In [52]:
def find_next_tile(current, current_edge, reverse, row):
    next_tile = [k for k in neighbors[current] if tuple(current_edge) in edges[k]]
    if len(next_tile) > 0:
        tiles[next_tile[0]] = [
            tile for tile in rotations(tiles[next_tile[0]]) 
            if tuple(current_edge) == (tuple(tile[:,-int(reverse)]) if row else tuple(tile[0]))
        ][0]
        return next_tile[0]
    return None

In [53]:
def get_next_tile(current, chart, chart_row, reverse, row=True):
    next_tile_id = find_next_tile(current, tiles[current][:,-(not reverse)] if row else tiles[current][-1], reverse, row)
    if next_tile_id:
        current = next_tile_id
        chart_row += [tiles[current][1:-1,1:-1]]
    else:
        chart += [np.hstack(chart_row)] if not reverse else [np.hstack(chart_row[::-1])]
        chart_row.clear()
    return next_tile_id

In [54]:
while current:
    next_tile_id = get_next_tile(current, chart, chart_row, reverse)
    if next_tile_id is None:
        reverse = not reverse
        current = get_next_tile(current, chart, chart_row, reverse, row=False)
    else:
        current = next_tile_id

ValueError: need at least one array to concatenate